<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/evidence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [4]:
import pickle

with open(f"{base_path}conversations_with_reasoning_media_removed.pkl", 'rb') as file:
    conversations = pickle.load(file)

print("Len loaded data:", len(conversations))


Len loaded data: 18184


In [28]:
def remove_characters_from_end(input_string):
    characters_to_remove = [')', '[', ')]', ').', '/', '/),']

    # Use rstrip to remove the specified characters from the end of the string
    for char in characters_to_remove:
        input_string = input_string.rstrip(char)

    return input_string

In [36]:
domains = set()
for key in conversations:
  for comment in conversations[key]['comments']:
    urls_and_domains = find_urls_and_domains(comment['body'])
    for url, domain in urls_and_domains:
            if domain and domain not in domains:
                domains.add(domain)

https://help2ukraine.org
https://stake.com
http://stakecasinos.online
https://headlinermusicclub.com
https://headlinermusicclub.com
https://bpmsupreme.com
https://bpmsupreme.com
https://translatedintoart.etsy.com
https://translatedintoart.etsy.com
https://www.saltyssalt.com
https://www.saltyssalt.com
https://fonts.gstatic.com
https://fonts.gstatic.com
https://freecoursesites.com
https://freecoursesites.com
https://nhost.io
https://nhost.io
https://rclshareholderbenefit.questionpro.com
https://rclshareholderbenefit.questionpro.com
https://www.cookie-dialog-monster.com
https://www.cookie-dialog-monster.com
https://www.i-dont-care-about-cookies.eu
https://www.i-dont-care-about-cookies.eu
https://killerskins.com
https://killerskins.com
http://ftroop.vk6flab.com
http://ftroop.vk6flab.com
https://framed.wtf
https://framed.wtf
https://moviedle.app
https://moviedle.app
http://uaithub.com
http://uaithub.com
http://uaithub.com
http://uaithub.com
https://www.santacruz4realsolutions.com
https://ww

In [38]:
len(domains)

4705

In [37]:
domains

{'www.christopherward.com',
 'd2h9y75tak3pkg.cloudfront.net',
 'www.soundtransit.org',
 'www.tide-forecast.com',
 'lastdayspast.com',
 'www.nps.gov',
 'eishauseats.com',
 'partedmagic.com',
 'sci-hub.se',
 'www.collegepravesh.com',
 'propsnation.blog',
 'www.5esrd.com',
 'paramountmovies.com',
 'withhive.me',
 'awardwallet.com',
 'facebook.com',
 'www.carbonbrief.org',
 'www.hakusensha.co.jp',
 'krakenbooks.net',
 'rutgers.ca1.qualtrics.com',
 'support.suretyhome.com',
 'www.bowiebible.com',
 'jksrestaurants.com',
 'www.preposterousuniverse.com',
 'www.curve.app',
 'www.gov.pl',
 'medium.com',
 'cdn-ssl.s7.disneystore.com',
 'daausa.org',
 'learnabout-electronics.org',
 'www.elmasacre.com',
 'www.michele4wakeschools.com)',
 'ads.reddit.com)',
 'www.cityofsantacruz.com',
 'she-raandtheprincessesofpower.fandom.com',
 'dev.epicgames.com',
 'www.hg.org',
 'www.badlandsoffroad.com',
 'headlinermusicclub.com',
 'www.leagueoflegends.com',
 'aa-intergroup.org',
 'www.udemy.com',
 'rpg.stackexc

In [29]:
from urllib.parse import urlparse
import re

def find_urls_and_domains(text):
    # Define a regex pattern to match URLs
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

    # Find all matches in the text
    urls = re.findall(url_pattern, text)

    # Extract domains from URLs using urlparse with error handling
    domains = []
    for url in urls:
        try:
            domain = urlparse(url).hostname
            domains.append((url, domain))
        except ValueError as e:
            result = url.split('](')
            for part in result:
                cleaned = remove_characters_from_end(part)
                print(cleaned)
                domain = urlparse(cleaned).hostname
                domains.append((cleaned, domain))

    return domains